## MCMC Convergence/Autocorrelation
This notebook runs a chain for the fiducial cosmology for about 10e^6 samples and looks at the autocorrelation time $\tau$ each 100 iterations. In general we want at least $50\tau$ samples. We will assume the autocorrelation time is similar for each cosmology. We can always choose a conservative estimate for the number of samples to ensure the chains converge.

In [14]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from getdist import plots, MCSamples
import getdist
from multiprocessing import Pool
from getdist import plots, MCSamples

import sys
import time
from cocoa_emu import *
from cocoa_emu.emulator import NNEmulator, GPEmulator
from cocoa_emu.data_model import LSST_3x2

import emcee
import time

from numpy import linalg
import scipy

%matplotlib notebook

In [15]:
### Functions
def plot_cosmo_contours(sample_list, legend_labels):
    names = ['logA', 'ns', 'H0', 'omegab', 'omegac']
    labels =  ['logA', 'ns', 'H0', 'omega b', 'omega c']
    
    cosmo_truth = [3.0675, 0.97, 69., 0.0228528, 0.1199772]
    
    truth_dict = {}
    for name, truth in zip(names, cosmo_truth):
        truth_dict[name] = truth
        
    getdist_samples = []
    for samples, legend_label in zip(sample_list, legend_labels):
        cosmo_samples = samples[:,:5]
        getdist_samples.append(MCSamples(samples=cosmo_samples,names = names, labels=labels, label=legend_label))
    
    g = plots.get_subplot_plotter()
    g.triangle_plot(getdist_samples, filled=True, markers=truth_dict)
    
    #plt.show()
    
def add_bias(bias_theta, datavector):
    for i in range(5):
        factor = (bias_theta[i] / bias_fid[i])**bias_mask[i]
        datavector = factor * datavector
    return datavector

def add_shear_calib(m, datavector):
    for i in range(5):
        factor = (1 + m[i])**shear_calib_mask[i]
        datavector = factor * datavector
    return datavector

def hard_prior(theta, params_prior):
    """
    A function to impose a flat prior on a set of parameters.
    :theta: The set of parameter values
    :params_prior: The minimum and the maximum value of the parameters on which this prior is imposed
    """
    is_lower_than_min = bool(np.sum(theta < params_prior[:,0]))
    is_higher_than_max = bool(np.sum(theta > params_prior[:,1]))
    if is_lower_than_min or is_higher_than_max:
        return -np.inf
    else:
        return 0.
    
cosmo_prior_lim = np.array([[1.61, 3.91],
                       [0.87, 1.07],
                       [55, 91],
                       [0.01, 0.04],
                       [0.001, 0.99]])

ia_prior_lim = np.array([[-5., 5.],
                       [-5., 5.]])

bias_prior_lim = np.array([[0.8, 3.],
                       [0.8, 3.],
                       [0.8, 3.],
                       [0.8, 3.],
                       [0.8, 3.]])

baryon_prior_lim = np.array([[-3., 12.],
                             [-2.5, 2.5]])

baryon_prior_lim = 3. * baryon_prior_lim 

dz_source_std   = 0.002 * np.ones(5)
dz_lens_std     = 0.005 * np.ones(5)
shear_calib_std = 0.005 * np.ones(5)
    
def lnprior(theta):
    cosmo_theta = theta[:5]
    ns          = cosmo_theta[1]

    ns_prior    = 0.
    
    dz_source   = theta[5:10]
    ia_theta    = theta[10:12]
    dz_lens     = theta[12:17]
    bias        = theta[17:22]
    shear_calib = theta[22:27]
    baryon_q    = theta[27:]
    
    cosmo_prior = hard_prior(cosmo_theta, cosmo_prior_lim) + ns_prior
    ia_prior    = hard_prior(ia_theta, ia_prior_lim)
    bias_prior  = hard_prior(bias, bias_prior_lim)
    baryon_prior = hard_prior(baryon_q, baryon_prior_lim)
    
    dz_source_lnprior   = -0.5 * np.sum((dz_source / dz_source_std)**2)
    dz_lens_lnprior     = -0.5 * np.sum((dz_lens / dz_lens_std)**2)
    shear_calib_lnprior = -0.5 * np.sum((shear_calib / shear_calib_std)**2)
    
    return cosmo_prior + ia_prior + dz_source_lnprior + dz_lens_lnprior + \
            shear_calib_lnprior + bias_prior + baryon_prior
    
def ln_lkl(theta):
    model_datavector = get_data_vector_emu(theta)
    delta_dv = (model_datavector - data_model.dv_obs)[data_model.mask_3x2]
    return -0.5 * delta_dv @ data_model.masked_inv_cov @ delta_dv

def lnprob(theta):
    return lnprior(theta) + ln_lkl(theta)

def get_data_vector_emu(theta):
    """
    Function to get the emulated data vector (including the effect of galaxy bias, baryons, etc.)
    """
    cosmo_ia_dz_theta = theta[:17]
    bias        = theta[17:22]
    shear_calib = theta[22:27]
    baryon_q    = theta[27:]
    datavector = data_model.compute_datavector(cosmo_ia_dz_theta)
    datavector = np.array(datavector)
    datavector = add_bias(bias, datavector)
    datavector = add_shear_calib(shear_calib, datavector)
    return datavector

In [7]:
N_WALKERS     = 120
N_MCMC        = 200000
N_SAMPLES     = N_WALKERS*N_MCMC
NDIM_SAMPLING = 29

In [8]:
### Get the necessary data for likelihood

# Get the LSST covariance and fid data
path = '/home/grads/data/evan/tension_calibration/data/lsst_y1/'
lsst_cov = np.loadtxt(path+'cov_lsst_y1')
fid_cos = np.loadtxt(path+'lsst_y1_data_fid',dtype=np.float32)[:,1]

lsst_y1_cov = np.zeros((1560, 1560))
for line in lsst_cov:
    i = int(line[0])
    j = int(line[1])

    cov_g_block  = line[-2]
    cov_ng_block = line[-1]

    cov_ij = cov_g_block + cov_ng_block

    lsst_y1_cov[i,j] = cov_ij
    lsst_y1_cov[j,i] = cov_ij
    
fid = torch.Tensor(fid_cos)
cov = torch.Tensor(lsst_y1_cov)

# Code taken from the emulator notebook
#first the fiducial cosmology

configfile = 'configs/nn_emu.yaml'
config = Config(configfile)

config_args     = config.config_args
config_args_io  = config_args['io']
config_args_data = config_args['data']

savedir = 'output/nn_emu/'

N_DIM         = 17
data_model    = LSST_3x2(N_DIM, config_args_io, config_args_data)
data_model.emu_type = 'nn'
OUTPUT_DIM = 1560

emu = NNEmulator(N_DIM, OUTPUT_DIM, data_model.dv_fid, data_model.dv_std)    
emu.load('model/nn_emu/model')
# ======================================================

data_model.emu = emu

bias_fid         = data_model.bias_fid
bias_mask        = data_model.bias_mask
shear_calib_mask = data_model.shear_calib_mask

theta0    = np.array([3.0675, 0.97, 69.0, 0.0228528, 0.1199772, 
                      0., 0., 0., 0., 0.,
                      0.5, 0.,
                      0., 0., 0., 0., 0.,
                      1.24, 1.36, 1.47, 1.60, 1.76,
                      0., 0., 0., 0., 0.,
                      0., 0.])

theta_std = np.array([0.01, 0.001, 0.1, 0.001, 0.002, 
                      0.002, 0.002, 0.002, 0.002, 0.002, 
                      0.1, 0.1,
                      0.005, 0.005, 0.005, 0.005, 0.005, 
                      0.03, 0.03, 0.03, 0.03, 0.03,
                      0.005, 0.005, 0.005, 0.005, 0.005, 
                      0.1, 0.1]) 

# Starting position of the emcee chain
pos0 = theta0[np.newaxis] + 3. * theta_std[np.newaxis] * np.random.normal(size=(N_WALKERS, NDIM_SAMPLING))

In [12]:
autocorr       = np.zeros(N_MCMC)
autocorr_ratio = np.zeros(N_MCMC)

index = 0
old_tau=np.inf
aold_tau=0

print('--- MCMC ---')
print('N_iterations = {}'.format(N_MCMC))
print('N_walkers = {}'.format(N_WALKERS))
print('N_samples = {}\n'.format(N_SAMPLES))

filename = "godzilla.h5"
backend = emcee.backends.HDFBackend(filename)
backend.reset(N_WALKERS, NDIM_SAMPLING) ### comment if you do not want the previous chain erased !!!!!!!!

with Pool(10) as pool:
    emu_sampler = emcee.EnsembleSampler(N_WALKERS, NDIM_SAMPLING, lnprob, pool=pool, backend=backend)
    emu_sampler.run_mcmc(pos0, N_MCMC, progress=True)
'''
    for sample in emu_sampler.sample(pos0, iterations=N_MCMC, progress=True):
        if(emu_sampler.iteration % 10000):
            continue

        # Compute the autocorrelation time so far
        # Using tol=0 means that we'll always get an estimate even
        # if it isn't trustworthy
        tau = emu_sampler.get_autocorr_time(tol=0)
        atau = np.mean(tau)
        autocorr[index] = atau
        autocorr_ratio[index] = atau/aold_tau
        index += 1

        old_tau = tau
        aold_tau = np.mean(old_tau)
'''

--- MCMC ---
N_iterations = 200000
N_walkers = 120
N_samples = 24000000



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200000/200000 [4:14:52<00:00, 13.08it/s]


"\n    for sample in emu_sampler.sample(pos0, iterations=N_MCMC, progress=True):\n        if(emu_sampler.iteration % 10000):\n            continue\n\n        # Compute the autocorrelation time so far\n        # Using tol=0 means that we'll always get an estimate even\n        # if it isn't trustworthy\n        tau = emu_sampler.get_autocorr_time(tol=0)\n        atau = np.mean(tau)\n        autocorr[index] = atau\n        autocorr_ratio[index] = atau/aold_tau\n        index += 1\n\n        old_tau = tau\n        aold_tau = np.mean(old_tau)\n"

In [13]:
'''
f, (ax1, ax2) = plt.subplots(1, 2, sharey=False)
n = 1000 * np.arange(2, index + 1)
print(n)
print(autocorr)

ax1.plot(n,autocorr[1:index])
ax1.plot(n,n/50,'k--',label='iterations/50')
ax1.set_ylabel('tau')
ax1.set_xlabel('iterations')
ax1.set_ylim(0,1250)
ax1.legend()
ax2.plot(n,autocorr_ratio[1:index])
ax2.plot([0,N_MCMC],[1,1],color='k')
ax2.plot([0,N_MCMC],[0.99,0.99],'k--')
ax2.plot([0,N_MCMC],[1.01,1.01],'k--')
ax2.set_xlabel('iterations')
ax2.set_ylabel('tau / prev. tau')
ax2.set_ylim(0.98,1.05)
#ax3.plot(n,autocorr_diff[1:index])
#ax3.set_ylim(-0.01,1.1)
'''

"\nf, (ax1, ax2) = plt.subplots(1, 2, sharey=False)\nn = 1000 * np.arange(2, index + 1)\nprint(n)\nprint(autocorr)\n\nax1.plot(n,autocorr[1:index])\nax1.plot(n,n/50,'k--',label='iterations/50')\nax1.set_ylabel('tau')\nax1.set_xlabel('iterations')\nax1.set_ylim(0,1250)\nax1.legend()\nax2.plot(n,autocorr_ratio[1:index])\nax2.plot([0,N_MCMC],[1,1],color='k')\nax2.plot([0,N_MCMC],[0.99,0.99],'k--')\nax2.plot([0,N_MCMC],[1.01,1.01],'k--')\nax2.set_xlabel('iterations')\nax2.set_ylabel('tau / prev. tau')\nax2.set_ylim(0.98,1.05)\n#ax3.plot(n,autocorr_diff[1:index])\n#ax3.set_ylim(-0.01,1.1)\n"

In [ ]:
#print(len(autocorr[:index]))

In [24]:
filename = "godzilla.h5"
reader = emcee.backends.HDFBackend(filename)

samples = reader.get_chain(flat=True)
print(samples)

[[ 3.07534296e+00  9.68475514e-01  6.87605433e+01 ...  1.16508062e-02
  -5.01119927e-01  1.10234833e-01]
 [ 3.06723652e+00  9.73372822e-01  6.89360831e+01 ...  1.54206219e-02
   1.91042641e-01 -6.34389417e-02]
 [ 3.04302129e+00  9.69618015e-01  6.93401232e+01 ... -1.22156168e-03
  -3.91580667e-01 -3.68599706e-01]
 ...
 [ 3.16067819e+00  9.99156048e-01  6.51764827e+01 ... -4.21112546e-03
  -1.50140945e+00 -5.67746438e+00]
 [ 3.01789088e+00  9.57851043e-01  7.67322775e+01 ...  1.37380611e-03
   2.15472479e+01 -2.23244786e+00]
 [ 3.06321669e+00  9.74909949e-01  6.70803599e+01 ... -2.34321228e-03
   8.69703376e+00  1.97733852e+00]]


In [30]:
autocorr = reader.get_autocorr_time()
print(autocorr)

[2341.99534268 2247.25560561 2240.18575022 2091.91863711 2338.91995198
  887.46367363  890.26328236  891.69624243  916.67869511  890.89165756
  918.93087129  873.74329085  902.10421134  926.57282835  914.66225069
  922.99498757  908.14471594  996.23355772  974.02796871  966.1156263
  969.17281602  964.48349499  890.6920013   886.77576723  852.15243163
  892.50844566  902.21546636 1023.86765241 1019.04366777]


In [31]:
print(np.mean(autocorr))

1153.162444511595
